In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import TargetEncoder
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import joblib

# Read the data
train_origin = pd.read_csv('/Users/jaesolshin/내 드라이브/2024-2/Google ML Bootcamp2024/data/playground1/train.csv')

In [ ]:
# 데이터 샘플링
#train = train_origin.sample(frac=0.001, random_state = 42).set_index('id').astype(str)
train = train_origin.set_index('id').astype(str)

# 예측변수 분리 및 train, valid set 분리
X = train.drop(['Response'], axis=1)
y = train['Response'].astype(float)

# Train/Test split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# 타겟 인코딩
enc = TargetEncoder()
X_train = enc.fit_transform(X_train, y_train)
X_valid = enc.transform(X_valid)  # 학습된 인코더를 테스트 데이터에 적용
X_train = X_train.astype(float)
X_valid = X_valid.astype(float)

In [ ]:
results=[]

def modeling(model):
    model.fit(X_train, y_train)
    valid_preds = model.predict_proba(X_valid)[:, 1]
    valid_auc = roc_auc_score(y_valid, valid_preds)
    print("ROC AUC:", valid_auc)

    results.append({"ROC AUC":valid_auc,"valid_preds":valid_preds})
    return valid_auc, valid_preds

LightGBM   ROC-AUC: 0.8854   Time: 2m 32.1s
{'lambda_l1': 0.1, 'lambda_l2': 0.1, 'learning_rate': 0.1, 'max_depth': -1, 'n_estimators': 100, 'num_leaves': 20, 'verbose':-1}



In [10]:
# LightGBM
print("\nLightGBM:")
best_param = {'lambda_l1': 0.1, 'lambda_l2': 0.1, 'learning_rate': 0.1, 'max_depth': -1, 'n_estimators': 100, 'num_leaves': 20, 'verbose':-1}
#{'objective': 'binary', 'boosting_type': 'gbdt', 'num_leaves': 31, 'learning_rate': 0.05, 'feature_fraction': 0.9, 'n_estimators': 100, 'device': 'gpu', 'gpu_platform_id': 0, 'gpu_device_id': 0, 'metric': 'auc'}
lgbm_model1 = LGBMClassifier(**best_param, random_state=42)

modeling(lgbm_model1)


LightGBM:
ROC AUC: 0.8854378058218718


(0.8854378058218718,
 array([2.69057586e-01, 1.41895106e-01, 3.62949571e-01, ...,
        2.42836373e-04, 3.91265241e-04, 9.96636466e-02]))

LightGBM   ROC-AUC: 0.8848  Time: 2m 20.4s
{'objective': 'binary', 'boosting_type': 'gbdt', 'num_leaves': 31, 'learning_rate': 0.05, 'feature_fraction': 0.9, 'n_estimators': 100, 'device': 'gpu', 'gpu_platform_id': 0, 'gpu_device_id': 0, 'metric': 'auc'}

In [11]:
# LightGBM
print("\nLightGBM:")
best_param = {'objective': 'binary', 'boosting_type': 'gbdt', 'num_leaves': 31, 'learning_rate': 0.05, 'feature_fraction': 0.9, 'n_estimators': 100, 'metric': 'auc'}
lgbm_model2 = LGBMClassifier(**best_param, random_state=42)

modeling(lgbm_model2)


LightGBM:
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Number of positive: 1131875, number of negative: 8071963
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.197368 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1188
[LightGBM] [Info] Number of data points in the train set: 9203838, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.122979 -> initscore=-1.964521
[LightGBM] [Info] Start training from score -1.964521
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
ROC AUC: 0.884788781984587


(0.884788781984587,
 array([0.26687232, 0.13061131, 0.32398892, ..., 0.00115292, 0.00118716,
        0.1151039 ]))

XGBoost   ROC-AUC: 0.8857  Time: 1m 34.6s
{'n_estimators': 50, 'max_depth': 12, 'eval_metric':'logloss'}
'use_label_encoder': False, 'tree_method':'gpu_hist'

In [12]:
# XGBoost
print("\nXGBoost:")
xgb_param = {'n_estimators': 50, 'max_depth': 12, 'eval_metric':'logloss'}
xgb_model = XGBClassifier(**xgb_param, random_state=42)
modeling(xgb_model)


XGBoost:


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [13:39:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


ROC AUC: 0.8856846852175086


(0.8856846852175086,
 array([3.1889963e-01, 1.0619615e-01, 2.8649637e-01, ..., 1.3015227e-04,
        1.6171826e-04, 9.8949939e-02], dtype=float32))

CatBoost   ROC-AUC: 0.8869   Time: 168m 30.2s
iterations:1000 -> 41m 9.1s
iterations:3000 -> 168m 30.2s
{ 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'class_names': [0, 1], 'learning_rate': 0.075, 'iterations': 3000, 'depth': 9, 'random_strength': 0, 'l2_leaf_reg': 0.5, 'max_leaves': 512, 'fold_permutation_block': 64, 'verbose': False, 'allow_writing_files': False }

'task_type': 'GPU'

In [13]:
# CatBoost
print("\nCatBoost:")
cat_param = { 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'class_names': [0, 1], 'learning_rate': 0.075, 'iterations': 3000, 'depth': 9, 'random_strength': 0, 'l2_leaf_reg': 0.5, 'max_leaves': 512, 'fold_permutation_block': 64, 'allow_writing_files': False, 'verbose':0}
cat_model = CatBoostClassifier(**cat_param, random_state=42)
modeling(cat_model)


CatBoost:
ROC AUC: 0.8869308470546298


(0.8869308470546298,
 array([3.19141099e-01, 1.17099170e-01, 3.33643198e-01, ...,
        1.41015313e-04, 3.43219978e-04, 1.05149323e-01]))

In [ ]:
ensemble soft voting : 0.8869

In [26]:
print(results)
results_df = pd.DataFrame(results)
ensemble_preds = np.mean(results['valid_preds'], axis=0)
ensemble_auc = roc_auc_score(y_valid, ensemble_preds)
print("ensemble ROC AUC:", ensemble_auc)

    ROC AUC                                        valid_preds
0  0.885438  [0.269057585848406, 0.14189510575872583, 0.362...
1  0.884789  [0.26687232020494717, 0.130611305217624, 0.323...
2  0.885685  [0.31889963, 0.10619615, 0.28649637, 5.4846005...
3  0.886931  [0.31914109888247305, 0.11709916978790279, 0.3...
ensemble ROC AUC: 0.8869234514462901


In [27]:
#test 데이터 로드
test_origin = pd.read_csv('/Users/jaesolshin/내 드라이브/2024-2/Google ML Bootcamp2024/data/playground1/test.csv')

#인덱스 제외
X_test = test_origin.set_index('id').astype(str)

# 변수 타겟 인코딩
X_test = pd.DataFrame(enc.transform(X_test), index=X_test.index, columns=X_test.columns)

In [38]:
classifiers = [
    {'model':lgbm_model1, 'name':'lgbm_model1'}, 
    {'model':lgbm_model2, 'name':'lgbm_model2'},
    {'model':xgb_model, 'name':'xgb_model'},
    {'model':cat_model, 'name':'cat_model'}
]

sub_results = []

for clf in classifiers:
    model = clf['model']
    name = clf['name']

    # 예측 생성
    y_test_pred = model.predict_proba(X_test)[:,1]
    clf['preds'] = y_test_pred

    # 'id'와 'Response' 열이 있는 DataFrame 생성
    submission = pd.DataFrame({'id': X_test.index, 'Response': y_test_pred})
    
    # 예측을 CSV 파일로 저장
    submission.to_csv(f'{name}_predictions.csv', index=False)
    print(f"Predictions saved to '{name}_predictions.csv'")


# 앙상블 예측 생성
ensemble_pred = np.mean([clf['preds'] for clf in classifiers], axis=0)
submission = pd.DataFrame({'id': X_test.index, 'Response': ensemble_pred})

# 예측을 CSV 파일로 저장
submission.to_csv('ensemble_predictions.csv', index=False)
print("Predictions saved to 'ensemble_predictions.csv'")

Predictions saved to 'lgbm_model1_predictions.csv'
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
Predictions saved to 'lgbm_model2_predictions.csv'
Predictions saved to 'xgb_model_predictions.csv'
Predictions saved to 'cat_model_predictions.csv'
Predictions saved to 'ensemble_predictions.csv'


In [ ]:
submision.head()

In [5]:
cd ..

/Users/jaesolshin/Documents/GitHub/kgpg_binary


/Users/jaesolshin/Library/Python/3.12/lib/python/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [17]:
import pandas as pd
submission1 = pd.read_csv('submission/cat_model_predictions.csv')
submission2 = pd.read_csv('submission/ensemble_predictions.csv')

# 'id'를 기준으로 두 DataFrame 병합
submission = pd.merge(submission1, submission2, on='id', suffixes=('_1', '_2'))

# 'Response' 열들의 평균을 계산하여 새로운 'Response' 변수 생성
submission['Response'] = submission[['Response_1', 'Response_2']].mean(axis=1)

# 원래 'Response' 변수 삭제
submission = submission.drop(columns=['Response_1', 'Response_2'])

# 예측을 CSV 파일로 저장
submission.to_csv('submission/cat_keras_predictions.csv', index=False)
print("Predictions saved to 'cat_keras_predictions.csv'")


Predictions saved to 'cat_keras_predictions.csv'


In [16]:
submission.head()

,id,Response
0,11504798,0.004916
1,11504799,0.624402
2,11504800,0.250594
3,11504801,0.000181
4,11504802,0.183111
